# Sheet Manager Usage

This notebook demonstrates the usage of the `SheetManager` class from the `snap_fit` package.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from collections.abc import Callable
from pathlib import Path

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.aruco.aruco_detector import ArucoDetector
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

In [ ]:
def create_aruco_loader() -> Callable[[Path], Sheet]:
    """Create a loader function that uses ArUco markers to rectify sheets."""
    # 1. Configure ArUco Board and Detector
    # Using defaults which match the printed board used for 'data/oca'
    board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
    detector_config = ArucoDetectorConfig()

    # Need a generator to create the board object for the detector
    board_generator = ArucoBoardGenerator(board_config)
    detector = ArucoDetector(board_generator, detector_config)

    # 2. Calculate crop margin to remove markers from the final image
    crop_margin = (
        board_config.marker_length + board_config.margin + detector_config.rect_margin
    )
    lg.debug(f"Computed crop margin: {crop_margin}")

    # 3. Initialize SheetAruco helper
    sheet_aruco = SheetAruco(detector, crop_margin=crop_margin)

    # 4. Define the loader function
    def loader(path: Path) -> Sheet:
        # SheetAruco.load_sheet handles loading, rectification, and Sheet creation
        return sheet_aruco.load_sheet(path)

    return loader


aruco_loader = create_aruco_loader()

In [ ]:
# Usage Example with Real Data

manager = SheetManager()

paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"

lg.info(f"Loading data from {data_dir}")
# Batch load images from data/oca
# This might take a moment as it processes images
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

# Verify Sheets
sheets = manager.get_sheets_ls()
print(f"Managed Sheets: {len(sheets)}")
for sheet in sheets:
    print(f" - {sheet.img_fp.name}: {len(sheet.pieces)} pieces")

# Verify Pieces
pieces = manager.get_pieces_ls()
print(f"Total Pieces: {len(pieces)}")

if pieces:
    print(f"Sample Piece: {pieces[0]}")

    # Optional: Visualize a piece if running in a notebook with display capabilities
    # import matplotlib.pyplot as plt
    # plt.imshow(pieces[0].img, cmap='gray')
    # plt.show()